# Pronóstico Semanal XGBoost — Comparativa Suavizado vs Original

Este cuaderno compara el desempeño del modelo entre dos escenarios:

1) **Original:** ventas tal cual (sin suavizado, con domingos incluidos)

2) **Suavizado:** si `Canti UMB = 0` (no domingo), reemplazar por el **promedio de los últimos 4** valores previos del **mismo día de la semana**.

Incluye:
- EDA previo global (Spearman + Información Mutua)
- EDA por bloque: **Avícola, Macro, Clima, Festividades** (top 5 + gráfico comparativo Spearman)
- Modelado con XGBoost (mismos hiperparámetros) y comparativa de métricas
- Importancias de variables (XGBoost)
- Conclusión visual


## 0) Configuración y librerías

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from xgboost import XGBRegressor
from math import sqrt

BASE_PATH = "."
FN_VENTAS = "Consolidado AWS.xlsx"
FN_MACRO  = "Macroeconomicas..xlsx"
FN_AVIC   = "Avicola..xlsx"
FN_CLIMA  = "Clima..xlsx"
FN_FEST   = "Festividades..xlsx"
SHEET_VENTAS = "Consolidado"
SHEET_FEST   = "Eventos_Diarios"

TEST_SIZE_PCT = 0.2
LAGS_TARGET   = [1,2,4,8]
ROLL_WINDOWS  = [4,8]

def fp(name): return str(Path(BASE_PATH) / name)

plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['axes.grid'] = True


## 1) Carga de datos

In [ ]:
aws  = pd.read_excel(fp(FN_VENTAS), sheet_name=SHEET_VENTAS)
macro = pd.read_excel(fp(FN_MACRO))
avic  = pd.read_excel(fp(FN_AVIC))
clima = pd.read_excel(fp(FN_CLIMA))
fest  = pd.read_excel(fp(FN_FEST), sheet_name=SHEET_FEST)

aws['Dia'] = pd.to_datetime(aws['Dia'])
if 'Fecha' in avic.columns:  avic['Fecha'] = pd.to_datetime(avic['Fecha'])
if 'fecha' in macro.columns: macro['fecha'] = pd.to_datetime(macro['fecha'])
if 'Fecha' in clima.columns: clima['Fecha'] = pd.to_datetime(clima['Fecha'])
fest['fecha'] = pd.to_datetime(fest['fecha'])

aws['Canti UMB'] = aws['Canti UMB'].astype(float)
print('Ventas shape:', aws.shape)


## 2) Preprocesamiento dual (Original vs Suavizado)

In [ ]:
aws_original = aws.copy()

aws_suav = aws.copy()
aws_suav['weekday'] = aws_suav['Dia'].dt.weekday
aws_suav = aws_suav.sort_values('Dia').reset_index(drop=True)
aws_suav['umb_nonzero'] = aws_suav['Canti UMB'].where(aws_suav['Canti UMB']>0, np.nan)
avg_last4 = aws_suav.groupby('weekday', group_keys=False)['umb_nonzero']\
                    .apply(lambda s: s.rolling(window=4, min_periods=1).mean().shift(1))
aws_suav['avg_last4_same_wd_prev'] = avg_last4
mask_zero = (aws_suav['Canti UMB'] == 0)
aws_suav.loc[mask_zero, 'Canti UMB'] = aws_suav.loc[mask_zero, 'avg_last4_same_wd_prev'].fillna(0.0).values
aws_suav.drop(columns=['weekday','umb_nonzero','avg_last4_same_wd_prev'], inplace=True)
print('Original:', aws_original.shape, '| Suavizado:', aws_suav.shape)


## 3) Agregación semanal y unión con exógenos

In [ ]:
def weekly_mean_numeric(df, date_col, prefix):
    d = df.copy(); d[date_col] = pd.to_datetime(d[date_col])
    num = d.select_dtypes(include='number')
    out = (pd.concat([d[[date_col]], num], axis=1).set_index(date_col)
           .resample('W').mean().add_prefix(prefix).reset_index())
    return out

def preparar_dataset(aws_df):
    ventas_diarias = aws_df.groupby('Dia')['Canti UMB'].sum().sort_index()
    ventas_sem = ventas_diarias.resample('W').sum().reset_index().rename(columns={'Dia':'Semana','Canti UMB':'ventas_sem'})
    avic_sem  = weekly_mean_numeric(avic, 'Fecha', 'avic_')  if 'Fecha' in avic.columns else pd.DataFrame()
    clima_sem = weekly_mean_numeric(clima,'Fecha','clima_')  if 'Fecha' in clima.columns else pd.DataFrame()
    macro_sem = weekly_mean_numeric(macro,'fecha','macro_')  if 'fecha' in macro.columns else pd.DataFrame()
    fest_sem = fest.groupby(pd.Grouper(key='fecha', freq='W')).agg('mean').reset_index()

    df = ventas_sem.copy()
    for block, key in [(avic_sem,'Fecha'), (clima_sem,'Fecha'), (macro_sem,'fecha'), (fest_sem,'fecha')]:
        if not block.empty and key in block.columns:
            df = df.merge(block, left_on='Semana', right_on=key, how='left').drop(columns=[key])
    return df.sort_values('Semana').reset_index(drop=True)

df_ori = preparar_dataset(aws_original)
df_sua = preparar_dataset(aws_suav)
print('Semanal original:', df_ori.shape, '| Semanal suavizado:', df_sua.shape)
df_ori.head(3)


## 4) EDA previo global (Spearman + Información Mutua)

In [ ]:
def eda_exog(df, label='ventas_sem'):
    num_df = df.select_dtypes(include='number').copy()
    num_df = num_df.fillna(num_df.mean(numeric_only=True))
    exog_cols = [c for c in num_df.columns if c != label]
    spear = num_df[exog_cols + [label]].corr(method='spearman')[label].drop(label)
    mi = pd.Series(mutual_info_regression(num_df[exog_cols], num_df[label], random_state=42), index=exog_cols)
    resumen = pd.DataFrame({'Spearman':spear, 'Mutual_Info':mi})
    resumen['|Spearman|'] = resumen['Spearman'].abs()
    resumen['Rank'] = (resumen['|Spearman|'].rank(ascending=False) + resumen['Mutual_Info'].rank(ascending=False))/2
    resumen = resumen.sort_values('Rank')
    return resumen, spear, mi

eda_ori, spear_ori, mi_ori = eda_exog(df_ori)
eda_sua, spear_sua, mi_sua = eda_exog(df_sua)

print('Top 10 exógenas (global) - Original'); display(eda_ori.head(10))
print('Top 10 exógenas (global) - Suavizado'); display(eda_sua.head(10))

top20 = eda_ori['|Spearman|'].sort_values(ascending=False).head(20).index.tolist()
corrmat = df_ori[top20 + ['ventas_sem']].fillna(df_ori.mean(numeric_only=True)).corr(method='spearman').abs()
plt.figure(figsize=(10,6))
plt.imshow(corrmat.values, aspect='auto')
plt.xticks(range(len(corrmat.columns)), corrmat.columns, rotation=90)
plt.yticks(range(len(corrmat.index)), corrmat.index)
plt.title('Heatmap Spearman (Top 20) - Dataset Original')
plt.colorbar()
plt.show()


## 5) Análisis por bloque (Avícola, Macro, Clima, Festividades) — Top 5 + Comparativa Spearman

In [ ]:
def top_block(resumen_df, prefix_list=None, contains_list=None, topk=5):
    idx = resumen_df.index
    mask = pd.Series(False, index=idx)
    if prefix_list:
        for p in prefix_list:
            mask = mask | idx.str.startswith(p)
    if contains_list:
        for s in contains_list:
            mask = mask | idx.str.contains(s)
    sub = resumen_df.loc[mask].copy()
    if sub.empty:
        return sub
    return sub.sort_values('Rank').head(topk)

def bar_compare_spearman(names, spear_A, spear_B, title):
    A = [spear_A.get(n, np.nan) for n in names]
    B = [spear_B.get(n, np.nan) for n in names]
    x = np.arange(len(names))
    width = 0.35
    plt.figure(figsize=(10,5))
    plt.bar(x - width/2, A, width, label='Original')
    plt.bar(x + width/2, B, width, label='Suavizado')
    plt.xticks(x, names, rotation=45, ha='right')
    plt.ylabel('Spearman')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()


### 5.1) 🐔 Avícola — Top 5 + Spearman (Original vs Suavizado)

In [ ]:
top_avic_ori = top_block(eda_ori, prefix_list=['avic_'], topk=5)
top_avic_sua = top_block(eda_sua, prefix_list=['avic_'], topk=5)
print('Top Avícola (Original)'); display(top_avic_ori)
print('Top Avícola (Suavizado)'); display(top_avic_sua)
names = top_avic_ori.index.tolist()
if names:
    bar_compare_spearman(names, spear_ori, spear_sua, 'Avícola: Spearman (Original vs Suavizado)')
else:
    print('No hay variables avícolas suficientes para graficar.')

### 5.2) 🏦 Macro — Top 5 + Spearman (Original vs Suavizado)

In [ ]:
top_macro_ori = top_block(eda_ori, prefix_list=['macro_'], topk=5)
top_macro_sua = top_block(eda_sua, prefix_list=['macro_'], topk=5)
print('Top Macro (Original)'); display(top_macro_ori)
print('Top Macro (Suavizado)'); display(top_macro_sua)
names = top_macro_ori.index.tolist()
if names:
    bar_compare_spearman(names, spear_ori, spear_sua, 'Macro: Spearman (Original vs Suavizado)')
else:
    print('No hay variables macro suficientes para graficar.')

### 5.3) 🌦️ Clima — Top 5 + Spearman (Original vs Suavizado)

In [ ]:
top_clima_ori = top_block(eda_ori, prefix_list=['clima_'], topk=5)
top_clima_sua = top_block(eda_sua, prefix_list=['clima_'], topk=5)
print('Top Clima (Original)'); display(top_clima_ori)
print('Top Clima (Suavizado)'); display(top_clima_sua)
names = top_clima_ori.index.tolist()
if names:
    bar_compare_spearman(names, spear_ori, spear_sua, 'Clima: Spearman (Original vs Suavizado)')
else:
    print('No hay variables de clima suficientes para graficar.')

### 5.4) 🎉 Festividades — Top 5 + Spearman (Original vs Suavizado)

In [ ]:
top_fest_ori = top_block(eda_ori, prefix_list=['tipo_evento_'], contains_list=['hay_evento'], topk=5)
top_fest_sua = top_block(eda_sua, prefix_list=['tipo_evento_'], contains_list=['hay_evento'], topk=5)
print('Top Festividades (Original)'); display(top_fest_ori)
print('Top Festividades (Suavizado)'); display(top_fest_sua)
names = top_fest_ori.index.tolist()
if names:
    bar_compare_spearman(names, spear_ori, spear_sua, 'Festividades: Spearman (Original vs Suavizado)')
else:
    print('No hay variables de festividades suficientes para graficar.')

## 6) Modelado XGBoost (ambos escenarios)

In [ ]:
def add_features(df):
    f = df.copy()
    for L in LAGS_TARGET: f[f'ventas_sem_lag{L}'] = f['ventas_sem'].shift(L)
    for w in ROLL_WINDOWS: f[f'ventas_sem_rm{w}'] = f['ventas_sem'].rolling(window=w, min_periods=1).mean()
    exog = [c for c in f.columns if c not in ['Semana','ventas_sem'] and pd.api.types.is_numeric_dtype(f[c])]
    for c in exog: f[f'{c}_lag1'] = f[c].shift(1)
    return f.dropna().reset_index(drop=True)

def fit_eval_xgb(df):
    df_fe = add_features(df)
    n = len(df_fe); split = int(n*(1-TEST_SIZE_PCT))
    train, test = df_fe.iloc[:split], df_fe.iloc[split:]
    X_train, y_train = train.drop(columns=['Semana','ventas_sem']), train['ventas_sem']
    X_test, y_test = test.drop(columns=['Semana','ventas_sem']), test['ventas_sem']
    y_pred_naive = test['ventas_sem_lag1']
    mae_naive = mean_absolute_error(y_test, y_pred_naive)
    rmse_naive = np.sqrt(mean_squared_error(y_test, y_pred_naive))
    model = XGBRegressor(n_estimators=600, learning_rate=0.05, max_depth=4,
                         subsample=0.85, colsample_bytree=0.85, reg_lambda=1.0, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mejora = (1 - mae/mae_naive)*100
    importancias = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    return {'MAE_Naive':mae_naive,'RMSE_Naive':rmse_naive,'MAE_XGB':mae,'RMSE_XGB':rmse,'Mejora%':mejora,'Importancias':importancias}

res_ori = fit_eval_xgb(df_ori)
res_sua = fit_eval_xgb(df_sua)

comp = pd.DataFrame({
    'Escenario':['Original','Suavizado'],
    'MAE_Naive':[res_ori['MAE_Naive'], res_sua['MAE_Naive']],
    'MAE_XGB':[res_ori['MAE_XGB'], res_sua['MAE_XGB']],
    'RMSE_XGB':[res_ori['RMSE_XGB'], res_sua['RMSE_XGB']],
    'Mejora%':[res_ori['Mejora%'], res_sua['Mejora%']]
})
print('Comparativa de desempeño (XGBoost):')
display(comp.style.format({'MAE_Naive':'{:.0f}', 'MAE_XGB':'{:.0f}', 'RMSE_XGB':'{:.0f}', 'Mejora%':'{:.1f}'}))

plt.figure()
plt.bar(comp['Escenario'], comp['MAE_XGB'])
plt.title('MAE XGBoost (Original vs Suavizado)')
plt.ylabel('MAE')
plt.show()


## 7) Importancias de variables (XGBoost)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,5))
res_ori['Importancias'].head(15).plot(kind='bar', ax=ax[0]); ax[0].set_title('Top 15 Importancias — Original'); ax[0].set_ylabel('Importancia')
res_sua['Importancias'].head(15).plot(kind='bar', ax=ax[1]); ax[1].set_title('Top 15 Importancias — Suavizado'); ax[1].set_ylabel('Importancia')
plt.tight_layout(); plt.show()


## 8) Conclusiones y variables exógenas recomendadas

In [ ]:
mejor = 'Original' if res_ori['MAE_XGB'] < res_sua['MAE_XGB'] else 'Suavizado'
print(f'Conclusión general: El escenario {mejor} muestra mejor desempeño global (menor MAE/RMSE).')

print('\nVariables exógenas recomendadas (consistentes en ambos escenarios):')
top10_ori = set(eda_ori.head(10).index)
top10_sua = set(eda_sua.head(10).index)
consistentes = list(top10_ori.intersection(top10_sua))
for v in sorted(consistentes):
    print('-', v)

print('\nTop 5 Avícola (Original):')
try:
    for v in top_avic_ori.index: print('-', v)
except NameError:
    print('N/D')
print('\nTop 5 Macro (Original):')
try:
    for v in top_macro_ori.index: print('-', v)
except NameError:
    print('N/D')
print('\nTop 5 Clima (Original):')
try:
    for v in top_clima_ori.index: print('-', v)
except NameError:
    print('N/D')
print('\nTop 5 Festividades (Original):')
try:
    for v in top_fest_ori.index: print('-', v)
except NameError:
    print('N/D')
